In [223]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'eps_study/join_fb_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/eps_study/join_fb_1/outputs/results/run']


In [224]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key', 'spec.index.search',
       'spec.index.type', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


# Reports

### Overall Duration

In [225]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration.columns)
print(overall_duration.index)
throughput = overall_duration
print(throughput.to_markdown())
throughput.plot()
'''
for column in throughput.columns:
    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])
    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0
print(throughput)
overall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() 
overall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() 
'''

Index(['flatpgm1024', 'flatpgm2048', 'flatpgm256', 'flatpgm4096',
       'flatpgm8192', 'sj'],
      dtype='object', name='spec.algo_name')
Index([1, 10, 100, 1000], dtype='int64', name='spec.common_key')
|   spec.common_key |   flatpgm1024 |   flatpgm2048 |   flatpgm256 |   flatpgm4096 |   flatpgm8192 |          sj |
|------------------:|--------------:|--------------:|-------------:|--------------:|--------------:|------------:|
|                 1 |   1.17219e+09 |   1.79725e+09 |  1.06313e+09 |   1.28248e+09 |   1.34946e+09 | 5.80793e+08 |
|                10 |   1.79187e+08 |   1.81795e+08 |  1.64653e+08 |   1.88454e+08 |   1.82988e+08 | 2.02129e+08 |
|               100 |   5.53367e+07 |   5.63119e+07 |  5.28592e+07 |   6.09474e+07 |   5.35882e+07 | 1.57892e+08 |
|              1000 |   2.68736e+07 |   2.35769e+07 |  1.55118e+07 |   3.90226e+07 |   3.95034e+07 | 1.50613e+08 |


'\nfor column in throughput.columns:\n    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])\n    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0\nprint(throughput)\noverall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() \noverall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() \n'

In [226]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
inner_index_total_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_total_bytes_fetched', aggfunc='median')
inner_index_fetch_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch_size', aggfunc='median')
print(inner_index_total_fetch)


spec.algo_name   flatpgm1024  flatpgm2048  flatpgm256  flatpgm4096  \
spec.common_key                                                      
1                  156066576    156066576   156066480    156066576   
10                 156065664    156066496   156040016    156066576   
100                155578288    156015680   147398016    156062896   
1000               107110400    134701056    58630144    150471248   

spec.algo_name   flatpgm8192         sj  
spec.common_key                          
1                  156066576  156066576  
10                 156066576  156066576  
100                156064992  156065792  
1000               155116528  156057600  


In [227]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 100])
print(result_checksum.loc[result_checksum['spec.common_key'] == 100]['result.checksum'].unique())

common_key: 1 checksum: ['D1870CC3EEAA3EE0262727AEC635A778'] OK
common_key: 10 checksum: ['9A10FE3267020EC7AE4F7CBC74860473'] OK
common_key: 100 checksum: ['0396A0E8E9B3A43C2FE141DF494ADB32'] OK
common_key: 1000 checksum: ['F952146E48E1C1B8212BB7CE2C5CFBFE'] OK
    spec.common_key spec.algo_name                   result.checksum
10              100    flatpgm1024  0396A0E8E9B3A43C2FE141DF494ADB32
20              100    flatpgm2048  0396A0E8E9B3A43C2FE141DF494ADB32
9               100     flatpgm256  0396A0E8E9B3A43C2FE141DF494ADB32
12              100    flatpgm4096  0396A0E8E9B3A43C2FE141DF494ADB32
19              100    flatpgm8192  0396A0E8E9B3A43C2FE141DF494ADB32
4               100             sj  0396A0E8E9B3A43C2FE141DF494ADB32
['0396A0E8E9B3A43C2FE141DF494ADB32']
